In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pyreadstat
import os

In [2]:
pd.set_option('display.max_columns', None)
os.chdir('C:/Users/511232/Desktop/MICS/microdata')
[f for f in os.listdir() if 'sav' in f]

['bh.sav', 'ch.sav', 'fs.sav', 'hh.sav', 'hl.sav', 'wm.sav']

In [52]:
os.chdir('C:/Users/511232/Desktop/MICS/microdata')
df_hh,meta_hh=pyreadstat.read_sav('hh.sav', apply_value_formats=False)
df_wm,meta_wm=pyreadstat.read_sav('wm.sav', apply_value_formats=False)
df_hl,meta_hl=pyreadstat.read_sav('hl.sav', apply_value_formats=False)

col_names_hh=meta_hh.column_names_to_labels
col_vals_hh=meta_hh.variable_value_labels
col_names_hl=meta_hl.column_names_to_labels
col_vals_hl=meta_hl.variable_value_labels
col_names_wm=meta_wm.column_names_to_labels
col_vals_wm=meta_wm.variable_value_labels

data_hh=df_hh.copy()
data_wm=df_wm.copy()
data_hl=df_hl.copy()

In [54]:
data_wm['disability'].value_counts()

2.0    9572
1.0     222
Name: disability, dtype: int64

In [102]:
'''data processing prior to generating crosstabs'''

class transform:

    def __init__(self):
        #reading in the .sav files and their metadata files
        os.chdir('C:/Users/511232/Desktop/MICS/microdata')
        df_hh,meta_hh=pyreadstat.read_sav('hh.sav', apply_value_formats=False)
        df_wm,meta_wm=pyreadstat.read_sav('wm.sav', apply_value_formats=False)
        df_hl,meta_hl=pyreadstat.read_sav('hl.sav', apply_value_formats=False)
        
        self.col_names_hh=meta_hh.column_names_to_labels
        self.col_vals_hh=meta_hh.variable_value_labels
        self.col_names_hl=meta_hl.column_names_to_labels
        self.col_vals_hl=meta_hl.variable_value_labels
        self.col_names_wm=meta_wm.column_names_to_labels
        self.col_vals_wm=meta_wm.variable_value_labels

        self.data_hh=df_hh.copy()
        self.data_wm=df_wm.copy()
        self.data_hl=df_hl.copy()

        self.disability_levels={1:'No difficulty',
        2:'Some difficulty',
        3:'A lot of difficulty',
        4:'Cannot do at all'}
        
        self.disability_cols=['AF6','AF8','AF9','AF10','AF11','AF12']
        self.other_cols=['WAGE','HH6','disability','windex5u','windex5r','windex5','MSTATUS','HC14']

        self.dis_names={'AF6': 'Difficulty seeing, even if wearing glasses or contact lenses',
        'AF8': 'Difficulty hearing, even if using a hearing aid',
        'AF9': 'Difficulty walking or climbing steps',
        'AF10': 'Difficulty remembering or concentrating',
        'AF11': 'Difficulty with self-care, such as washing all over or dressing',
        'AF12': 'Difficulty communicating'}

    def process_data(self):

        os.chdir('C:/Users/511232/Desktop/MICS/Crosstabs')
        ###################### VARIABLE CREATION & MERGES #####################################
        ################create hh_type####################################
        #calculate hh_type variable from HL3 from dataframe df_hl
        def family_type(df):
            nuclear=[1,2,3,13]
            extended=nuclear+[4,5,6,7,8,9,10,11,12]
            composite=extended+[14,96,98]

            if all(df['HL3'].isin(nuclear)):
                df['hh_type']='Nuclear'
            elif all(df['HL3'].isin(extended)):
                df['hh_type']='Extended'
            elif all(df['HL3'].isin(composite)):
                df['hh_type']='Composite'
            else:
                df['hh_type']='Unknown'
            return(df)
        
        #create hh_type
        self.data_hl=self.data_hl.groupby(['HH1','HH2']).apply(family_type)

        #merge data_hh to data_wm on [HH1,HH2] to add HC14
        right_df=self.data_hh[['HH1','HH2','HC14']]
        left_df=self.data_wm
        self.data_wm=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])

        #create hh_size
        self.data_hh['hh_size']=np.where(process.data_hh['HH48']>=8, '8+',process.data_hh['HH48'])
        #create living_alone
        cond=[self.data_hh['HH48']==1,self.data_hh['HH48'].isna(), self.data_hh['HH48']>1]
        result=['alone','Missing','not alone']
        self.data_hh['living_alone']=np.select(cond,result)
        #merge data_hh with data_hl to get 'hh_size','living_alone'
        right_df=self.data_hh[['HH1','HH2','hh_size','living_alone']]
        left_df=self.data_hl
        self.df_hl=pd.merge(left_df,right_df, how='left',on=['HH1','HH2'])
        
        #merge data_hl with data_wm to add 'hh_size','HL3'(household head relation),'HL6'(age),'hh_type'
        right_df=self.df_hl[['HH1','HH2','HL1','hh_size','HL3','HL6','hh_type','living_alone']]
        left_df=self.data_wm
        self.data_wm=pd.merge(left_df,right_df, how='left', 
        left_on=['HH1','HH2','LN'], right_on=['HH1','HH2','HL1'])

        #create 'disability_combined' variable. takes the max(code) among ['AF6','AF8','AF9','AF10','AF11','AF12']
        self.data_wm['disability_combined']=self.data_wm[self.disability_cols].apply(lambda x: x.max(), axis=1)
        self.data_wm['disability_combined']=self.data_wm['disability_combined'].map(self.disability_levels)
        #create head of household relationship variable as 1:HH 2:Other 
        self.data_wm['hh_rel']=np.where(self.data_wm['HL3']==1,1,2)
        self.data_wm['hh_rel']=self.data_wm['hh_rel'].map({1:'Head of household', 2:'Other'})

        ###################### LABEL VALUES #########################################
        for col in self.other_cols:
            if col in self.col_vals_hh.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_hh[col])
                print(f'{col} codes are translated from meta hh')
            elif col in self.col_vals_wm.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_wm[col])
                print(f'{col} codes are translated from meta women')
            elif col in self.col_vals_hl.keys():
                self.data_wm[col]=self.data_wm[col].map(self.col_vals_hl[col])
                print(f'{col} codes are translated from meta hhl')
            else:
                print(f'!!! WARNING !!! {col} codes were not translated')

In [103]:
process=transform()
process.process_data()

WAGE codes are translated from meta women
HH6 codes are translated from meta hh
disability codes are translated from meta women
windex5u codes are translated from meta hh
windex5r codes are translated from meta hh
windex5 codes are translated from meta hh
MSTATUS codes are translated from meta women
HC14 codes are translated from meta hh


In [115]:
'''Table 1
steps:
'disability_combined' column is calculated by taking the max(code) among ['AF6','AF8','AF9','AF10','AF11','AF12']
'''

def combined_disabilities(age_disaggregated=1):
    
    df=process.data_wm.copy()
    #crosstab
    if age_disaggregated:
        xtab=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],df['WAGE'],
        rownames=['Area','Disability','Disability level'],colnames=['Age'], values=df['wmweight'], aggfunc='sum',dropna=False)      
        #export as excel
        xtab.to_excel('xtab_all_dis_ByAge.xlsx')
    else:
        xtab=pd.crosstab([df['disability'],df['disability_combined']],df['HH6'],
        rownames=['Disability','Disability level'],colnames=['Area'], values=df['wmweight'], aggfunc='sum',dropna=False)
        #export as excel
        xtab.to_excel('xtab_all_dis_ByTotalAge.xlsx')

In [116]:
combined_disabilities(age_disaggregated=1)
combined_disabilities(age_disaggregated=0)

In [101]:
'''Table 2
steps:
-generate separate xtabs for all disability_cols
-stack() them to have a multiindex series and add them to a generator
-concatenate the generator items
-stack() and unstack() to get to the final result 
'''

def separate_disabilities():
        
    #will generate a list of multiindex series for each disability
    #generate a crosstab then stack to make it a multiindex series and put them 
    #all in a generator
    df=process.data_wm.copy()
    def xtab():
        for col in process.disability_cols:
            print(f'processing column {col}')
            #translate the codes
            df[col]=df[col].map(process.disability_levels)
            r=pd.crosstab([df['HH6'],df['disability'],df[col]],df['WAGE'],\
                rownames=['Area','Disability','Level'],colnames=['Age'], values=df['wmweight'], aggfunc='sum').stack()
            r.name=process.dis_names[col]
            yield(r)

    #concatenating the series in the resulting generator
    s=xtab()
    t=pd.concat(s, axis=1)
    t['All_disabilities']=t.sum(axis=1)

    #reshape the result
    T=t.stack().unstack([4,3]).sort_index(axis=1, level=0)
    T.to_excel('separate disabilites.xlsx')

In [102]:
separate_disabilities()

processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12


In [105]:
'''Table 4
steps:
-calculate domain_num by summing the True over the array of disability_cols values
if the array contains codes (3-a lot of difficulty) or (4-cannot at all) it will reult as True
'''
def num_dis_domain():
    
    #for each row under disability_cols if the row contains 3 or 4 then True
    #sum over all the True/False results 
    df=process.data_wm.copy()
    df['domain_num']=df[process.disability_cols].apply(lambda x: sum(x.isin([3,4])), axis=1)
    #generate xtab
    r=pd.crosstab([df['HH6'],df['disability']],df['domain_num'],\
        rownames=['Area','Disability'],colnames=['Number of domains'], values=df['wmweight'], aggfunc='sum', dropna=False)
    
    r.to_excel('Number_dis_domain.xlsx')

In [106]:
num_dis_domain()

In [34]:
'''Table 5 marital status'''

def marital_status():
    
    df=process.data_wm.copy()
    
    #crosstab
    xtab=pd.crosstab([df['HH6'],df['MSTATUS'],df['disability'],df['disability_combined']],df['WAGE'],
    rownames=['Area','Marital status','Disability','Disability level'],colnames=['Age'], values=df['wmweight'],
    aggfunc='sum',dropna=False)      
    #export as excel
    xtab.to_excel('MaritalStatus.xlsx')

In [35]:
marital_status()

In [109]:
'''Table 6: head_HH 2 crosstabs 
1-disability against head of household and othery type of relationship
steps:
-create head of household relationship (in the process_data_wm() )
df['hh_rel']=np.where(df['HL3']==1,1,2) where 1:HH 2:Other 
2-disability by head of households by wealth quintiles
steps
-will generate crosstab among disabled HH with wealth quintiles 
using windex and not specific windex5u for urban and windex5r for rural since
they differ from windex5 and will produce contradictory results between urban and rural xtabs
and the urban and rural disaggregation in the xtab for the total: camp/urban/rural
'''
def head_HH(quintile=0):
    
    df=process.data_wm.copy()
    if not quintile:
        #crosstab
        xtab=pd.crosstab([df['HH6'],df['disability'],df['disability_combined']],df['hh_rel'],
        rownames=['Area','Disability','Disability level'],colnames=['HH relationship'], values=df['wmweight'],
        aggfunc='sum',dropna=False)

        #export as excel
        xtab.to_excel('head of HH.xlsx')
    else:
        #filter out the HH
        df_hh_only=df[df['hh_rel']=='Head of household']
        #crosstab
        xtab=pd.crosstab([df_hh_only['HH6'],df_hh_only['disability'],df_hh_only['disability_combined']],df_hh_only['windex5'],
        rownames=['Area','Disability','Disability level'],colnames=['wealth quintile'], values=df_hh_only['wmweight'],
        aggfunc='sum',dropna=False)
        xtab.to_excel('head of HH_with wquintile.xlsx')


In [110]:
head_HH(quintile=0)
head_HH(quintile=1)

In [111]:
'''Table 7: Poorest_type
steps
-filter out the poorest quintile 'windex5' and crosstab with all disability types
-loop over disability_cols and create crosstabs then stack to end up with multiindex series
-put them in a generator and concatenate the generator items
'''

def poorest_type():
    
    df=process.data_wm.copy()
    #filter out the poorest
    df_poorest=df[df['windex5']=='Poorest'].copy()

    #will generate a list of multiindex series for each disability
    #generate a crosstab then stack to make it a multiindex series and put them 
    #all in a generator
    def xtab():
        for col in process.disability_cols:
            print(f'processing column {col}')
            #translate the codes
            df_poorest[col]=df_poorest[col].map(process.disability_levels)
            r=pd.crosstab([df_poorest['HH6'],df_poorest['disability']],df_poorest[col],\
                rownames=['Area','Disability'],colnames=['Disability level'], values=df_poorest['wmweight'], aggfunc='sum').stack()
            r.name=process.dis_names[col]
            yield(r)

    #concatenating the series in the resulting generator
    s=xtab()
    t=pd.concat(s, axis=1)
    t['All_disabilities']=t.sum(axis=1)
    t.to_excel('poorest_type.xlsx')

In [112]:
poorest_type()

processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12


In [113]:
'''Table 8: HH_type&size
Households with one or more persons with disabilities (18 years and older), by location and type and size of household
steps
-data_wm will be filtered according to (age>=18 & disability_combined==3,4) 
-get the 'HH1','HH2' of the resulting dataframe as a list by zipping both columns
-filter data resulting from  process_data_wm() on the tuple ('HH1','HH2')

steps for calculating type of household hh_type (in hl dataframe):
-grouby hl by ['HH1','HH2']
-if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter)
if ALL TRUE then code hh_type as 1 Nuclear
-if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter,
4 son /daughter in law, 5 grnachild, 6 parent, 7 parentin law, 8 brother/sister,
9, brother/sis in law, 10 uncle/aunt, 11 nephew/niece, 12 other)
if ALL TRUE then code hh_type as 2 Extended
if HL3 isin (1 head,2 spouse/partner,3 son/daughter,13 adopted son daughter,
4 son /daughter in law, 5 grandchild, 6 parent, 7 parent in law, 8 brother/sister,
9, brother/sis in law, 10 uncle/aunt, 11 nephew/niece, 12 other, 14 servant, 96 other, 98 dont know)
if ALL TRUE then code hh_type as 3 composite
WARNING: there is no way to distinguish two nuclear families in a single household from one
since for example a HH might have 2 spouses or more 
-data_wm with merge hl left_on=['HH1','HH2','LN'], right_on=['HH1','HH2','HL1'])
to get the hh_type variable
-perform corsstab
'''

def HH_type_size():

    df_w=process.data_wm.copy()
    #criteria 1 for being disabled, and criteria 2 for being >=18
    criteria1=((df_w['disability_combined']=='Cannot do at all')|(df_w['disability_combined']=='A lot of difficulty'))
    criteria2=(df_w['HL6']>=18)

    #filter according to criteria1 & criteria2
    df_filtered=df_w.loc[criteria1 & criteria2, ['HH1','HH2']].drop_duplicates()
    # filter df_w according to resulting ['HH1','HH2']
    hhd_filter=pd.Series(zip(df_w['HH1'],df_w['HH2'])).isin(list(zip(df_filtered['HH1'],df_filtered['HH2'])))
    #filter according to tuple ('HH1','HH2')
    df_wm_filtered=df_w[hhd_filter]

    ########################################################################
    #generate the crosstab
    xtab=pd.crosstab([df_wm_filtered['HH6'],df_wm_filtered['hh_type']],
    [df_wm_filtered['disability'],df_wm_filtered['disability_combined']],
    rownames=['Area','Household type'],colnames=['Disability','Disability level'], values=df_wm_filtered['wmweight'],
    aggfunc='sum',dropna=False)

    xtab.to_excel('hh_type_size1.xlsx')

    xtab=pd.crosstab([df_wm_filtered['HH6'],df_wm_filtered['hh_type']],df_wm_filtered['hh_size'],
    rownames=['Area','Household type'],colnames=['Household size'], values=df_wm_filtered['wmweight'],
    aggfunc='sum',dropna=False)

    xtab.to_excel('hh_type_size2.xlsx')


In [114]:
HH_type_size()

In [104]:
'''Table 9: living_type_age
rows: location(HH6), living_alone, hhd_inst (living alone, living with a family in hhd, living in institution)
note: Palestine doesnt have a question for place of hh whether institution or not so in this case
the hh_size is being used as alone versus not alone), disability (filter on disabled), disability combined
columns: separate disabilities (that is on ['AF6','AF8','AF9','AF10','AF11','AF12']), agegroups (WAGE)'''

def living_type_age():
    
    df_w=process.data_wm.copy()
    #criteria 1 for being disabled, and criteria 2 for being >=18
    criteria=((df_w['disability_combined']=='Cannot do at all')|(df_w['disability_combined']=='A lot of difficulty'))

    #filter according to criteria1 & criteria2
    df_filtered=df_w[criteria].copy()

    #since we have 2 levels of columns, use stack() twice
    def xtab():
        for col in process.disability_cols:
            print(f'processing column {col}')
            #translate the codes
            df_filtered[col]=df_filtered[col].map(process.disability_levels)
            #generate the crosstab
            r=pd.crosstab([df_filtered['HH6'],df_filtered['living_alone'],df_filtered['disability'],df_filtered['disability_combined']],
            [df_filtered['WAGE']],
            rownames=['Area','Living alone','Disability','Disability level'],colnames=['Age group'], values=df_filtered['wmweight'],
            aggfunc='sum',dropna=False).stack()
            r.name=process.dis_names[col]
            yield(r)

    #concatenating the series in the resulting generator
    s=xtab()
    t=pd.concat(s, axis=1)

    #reshape the result
    T=t.unstack([4]).sort_index(axis=1, level=0)
    T.to_excel('living_type_age.xlsx')


In [105]:
living_type_age()

processing column AF6
processing column AF8
processing column AF9
processing column AF10
processing column AF11
processing column AF12


In [ ]:
'''Table 10: house_ownership 
(House ownership of population (18 years and older) living alone, by sex, location and disability status)
filter data_wm on 'living_alone' and 'WAGE'>=18
Rows: location (HH6), disability (disability), disability combined
columns: house ownership (HC14: {1.0: 'OWN', 2.0: 'RENT', 6.0: 'OTHER', 9.0: 'NO RESPONSE'})'''

In [106]:
list(process.data_wm.columns)

['HH1',
 'HH2',
 'LN',
 'WM1',
 'WM2',
 'WM3',
 'WMINT',
 'WM4',
 'WM5',
 'WM6D',
 'WM6M',
 'WM6Y',
 'WM8',
 'WM9',
 'WM17',
 'WM7H',
 'WM7M',
 'WM10H',
 'WM10M',
 'WM11',
 'WMHINT',
 'WMFIN',
 'WB3M',
 'WB3Y',
 'WB4',
 'WB5',
 'WB6A',
 'WB6B',
 'WB7',
 'WB9',
 'WB10A',
 'WB10B',
 'WB11',
 'WB12A',
 'WB12B',
 'WB14',
 'WB15',
 'WB16',
 'WB17',
 'WB18',
 'WB19E',
 'WB19F',
 'WB19G',
 'WB19H',
 'WB19X',
 'WB19NR',
 'MT1',
 'MT2',
 'MT3',
 'MT4',
 'MT5',
 'MT6A',
 'MT6B',
 'MT6C',
 'MT6D',
 'MT6E',
 'MT6F',
 'MT6G',
 'MT6H',
 'MT6I',
 'MT9',
 'MT10',
 'MT11',
 'MT12',
 'CM1',
 'CM2',
 'CM3',
 'CM4',
 'CM5',
 'CM6',
 'CM7',
 'CM8',
 'CM9',
 'CM10',
 'CM11',
 'CM12',
 'CM15',
 'CM17',
 'BH11',
 'DB2',
 'DB4',
 'MN2',
 'MN3A',
 'MN3B',
 'MN3X',
 'MN3NR',
 'MN4AU',
 'MN4AN',
 'MN5',
 'MN6A',
 'MN6B',
 'MN6C',
 'MN19A',
 'MN19B',
 'MN19H',
 'MN19X',
 'MN19Y',
 'MN19NR',
 'MN20',
 'MN21',
 'MN22',
 'MN23',
 'MN24',
 'MN25',
 'MN26U',
 'MN26N',
 'MN32',
 'MN33',
 'MN34A',
 'MN34',
 'MN35',
 'MN3